<a href="https://colab.research.google.com/github/gbonanno/gab-dataart-challenge/blob/developer-gbonanno/src/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Engineer Challenge

En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

## Imports and initial settings

In [2]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 5.2 MB/s eta 0:00:00


In [3]:
pip install memory_profiler

In [13]:
# Imports
import pandas
import os
import json
import re
import emoji
from google.colab import drive
import gdown
import zipfile
from collections import defaultdict
import importlib.util
import sys
from memory_profiler import memory_usage
import cProfile

In [19]:
# Replace 'FILE_ID' with the actual file ID from the Google Drive link
file_id = '1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis'
file_path_zip = 'farmers-protest-tweets-2021-2-4.json.zip'
file_name = 'farmers-protest-tweets-2021-2-4.json'
local_path = '.'

gdown.download(f'https://drive.google.com/uc?id={file_id}', file_path_zip, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis
From (redirected): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&confirm=t&uuid=1f368019-88c0-4de2-829a-72c8ba6beb85
To: /content/farmers-protest-tweets-2021-2-4.json.zip
100%|██████████| 60.4M/60.4M [00:00<00:00, 96.8MB/s]


'farmers-protest-tweets-2021-2-4.json.zip'

In [20]:
# Descomprimir archivo
with zipfile.ZipFile(file_path_zip, 'r') as zip_ref:
    zip_ref.extractall(local_path)

In [22]:
# Se abre el archivo
fp = open(local_path + '/' + file_name, 'r+')

In [23]:
# Se bajan los tweets
# ENHANCEMENT - Extraer sólo los campos necesarios: date, user.username, content, mentionedUsers.username
tweets = []
for line in fp:
  tweets.append(json.loads(line))

In [24]:
# Revisar cuántos tweets son
len(tweets)

117407

In [25]:
# Imprimir un tweet para revisar el formato
line_json = json.loads(line)
print(json.dumps(line_json, indent=4))

{
    "url": "https://twitter.com/SikhVibes/status/1360040127146430470",
    "date": "2021-02-12T01:36:49+00:00",
    "content": "@Kisanektamorcha We are with you, keep the morcha alive and strong \ud83d\udcaa #FarmersProtest #MahapanchayatRevolution",
    "renderedContent": "@Kisanektamorcha We are with you, keep the morcha alive and strong \ud83d\udcaa #FarmersProtest #MahapanchayatRevolution",
    "id": 1360040127146430470,
    "user": {
        "username": "SikhVibes",
        "displayname": "SikhVibes",
        "id": 1568618503,
        "description": "SikhVibes.com is a premium Sikh Multimedia website with thousands of rare Audio Recordings, Videos and Katha from all over the world!",
        "rawDescription": "https://t.co/YWcGuzCBWn is a premium Sikh Multimedia website with thousands of rare Audio Recordings, Videos and Katha from all over the world!",
        "descriptionUrls": [
            {
                "text": "SikhVibes.com",
                "url": "http://SikhVibes.co

## Define function to get emojis

In [26]:
# String de testing
string_aux = "#FarmersProtest \n#ModiIgnoringFarmersDeaths \n#ModiDontSellFarmers \n@Kisanektamorcha \nFarmers constantly distroying crops throughout India. \nReally, it's hearts breaking...we care about our crops like our children. And govt. agriculture minister is laughing on us🚜🌾WE WILL WIN💪 🚜 https://t.co/kLspngG9xE"
string_aux

"#FarmersProtest \n#ModiIgnoringFarmersDeaths \n#ModiDontSellFarmers \n@Kisanektamorcha \nFarmers constantly distroying crops throughout India. \nReally, it's hearts breaking...we care about our crops like our children. And govt. agriculture minister is laughing on us🚜🌾WE WILL WIN💪 🚜 https://t.co/kLspngG9xE"

In [27]:
# Se define la primera versión de la función usando una regex.
# ISSUE: si dos emojis están contiguos (sin ningún caracter en el medio), lo considera un sólo emoji.
def extract_emojis(text):
  emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"  # Dingbats
    u"\U000024C2-\U0001F251"  # Enclosed characters
    u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "]+",
    flags=re.UNICODE
  )
  return emoji_pattern.findall(text)

In [28]:
# Probamos la función con el ejemplo.
extract_emojis(string_aux)

['🚜🌾', '💪', '🚜']

In [29]:
# Se define la segunda versión (por ahora la final) de la función usando la librería emoji.
# Esta función además cuenta cuántas veces aparece el emoji.
def extract_emojis2(text):
  emoji_count = defaultdict(int)
  for char in text:
    if char in emoji.EMOJI_DATA:
      emoji_count[char] += 1
  return emoji_count

In [30]:
# Probamos la función con el ejemplo.
extract_emojis2(string_aux)

defaultdict(int, {'🚜': 2, '🌾': 1, '💪': 1})

## Memory usage and execution time

In [31]:
# Medimos el uso de memoria de la función.
mem_usage = memory_usage((extract_emojis2, (string_aux,)))
print(f"Uso de memoria: {mem_usage}")

Memory usage: [1347.41015625, 1347.41015625, 1347.41015625]


In [32]:
# Medimos el tiempo de ejecución de la función.
cProfile.run('extract_emojis2(string_aux)')

         4 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-29-b73180df06de>:3(extract_emojis2)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Get functions from GitHub

In [33]:
# Descargar las funciones desde GitHub
drive.mount('/content/drive')
!git clone https://github.com/gbonanno/gab-dataart-challenge.git

Mounted at /content/drive
Cloning into 'gab-dataart-challenge'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 572.56 KiB | 1.74 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [41]:
# Leer las funciones y ejecutarlas
module_path = './gab-dataart-challenge/src/'
module_name1 = 'q1_memory'
module_name2 = 'q1_time'
module_name3 = 'q2_memory'
module_name4 = 'q2_time'
module_name5 = 'q3_memory'
module_name6 = 'q3_time'

# Cargar módulos
spec_q1_memory = importlib.util.spec_from_file_location(module_name1, module_path + module_name1 + '.py')
module_q1_memory = importlib.util.module_from_spec(spec_q1_memory)
sys.modules[module_name1] = module_q1_memory
spec_q1_memory.loader.exec_module(module_q1_memory)

spec_q1_time = importlib.util.spec_from_file_location(module_name2, module_path + module_name2 + '.py')
module_q1_time = importlib.util.module_from_spec(spec_q1_time)
sys.modules[module_name2] = module_q1_time
spec_q1_time.loader.exec_module(module_q1_time)

spec_q2_memory = importlib.util.spec_from_file_location(module_name3, module_path + module_name3 + '.py')
module_q2_memory = importlib.util.module_from_spec(spec_q2_memory)
sys.modules[module_name3] = module_q2_memory
spec_q2_memory.loader.exec_module(module_q2_memory)

spec_q2_time = importlib.util.spec_from_file_location(module_name4, module_path + module_name4 + '.py')
module_q2_time = importlib.util.module_from_spec(spec_q2_time)
sys.modules[module_name4] = module_q2_time
spec_q2_time.loader.exec_module(module_q2_time)

spec_q3_memory = importlib.util.spec_from_file_location(module_name5, module_path + module_name5 + '.py')
module_q3_memory = importlib.util.module_from_spec(spec_q3_memory)
sys.modules[module_name5] = module_q3_memory
spec_q3_memory.loader.exec_module(module_q3_memory)

spec_q3_time = importlib.util.spec_from_file_location(module_name6, module_path + module_name6 + '.py')
module_q3_time = importlib.util.module_from_spec(spec_q3_time)
sys.modules[module_name6] = module_q3_time
spec_q3_time.loader.exec_module(module_q3_time)

# Cargar las funciones desde los módulos
q1_memory = module_q1_memory.q1_memory
q1_time = module_q1_time.q1_time
q2_memory = module_q2_memory.q2_memory
q2_time = module_q2_time.q2_time
q3_memory = module_q3_memory.q3_memory
q3_time = module_q3_time.q3_time

In [42]:
# Llamada a la función 1
q1_memory(local_path + '/' + file_name)